# Decoder Based Transformer (GPT-2) Sampling
> Empirical netbook to sample bart for method2test benchmark.

In [1]:
from pathlib import Path
import csv
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import functools

pd.options.display.float_format = '{:.2f}'.format

In [2]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-07-13 20:40:31.282915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 20:40:31.492777: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import warnings
from matplotlib import colors
import os

In [4]:
def param_default():
    return {
        #'dataset' : 'codeparrot/codeparrot-clean-valid', #Deprecated
        'galeras_path' : '/workspaces/code-rationales/semeru-datasets/semeru/galeras/code_rationales',
        #'dataset' : 'code_completion_random_cut_5k_30_512_tokens',
        #'dataset' : 'code_completion_docstring_signature_5k_30_512_tokens',
        #'dataset' : 'code_completion_docstring_random_cut_5k_30_512_tokens',
        'dataset' : 'code_completion_docstring_5k_30_512_tokens',
        'dataset_disk_path': '/workspaces/code-rationales/semeru-datasets/codeparrot-clean-valid',
        'model_name': '/workspaces/code-rationales/data/codeparrot-small/checkpoints/checkpoint-29000', 
        'cache_dir': '/workspaces/code-rationales/datax/df_cache_dir', 
        'sampling_results': '/workspaces/code-rationales/data/sampling/gpt',
        'num_samples' : 100
    }

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data Loading and Testing

In [6]:
# Save dataset in nfs
#raw_datasets = load_dataset(param_default()['dataset'], cache_dir=param_default()['cache_dir'])
#raw_datasets.save_to_disk(param_default()['dataset_disk_path'])

In [7]:
# Reload with the `json` script
#test_dataset = load_from_disk(param_default()['dataset_disk_path'])
#test_dataset

In [8]:
test_dataset = load_dataset('json',data_files=param_default()['galeras_path'] + '/' + param_default()['dataset'] + '.json', cache_dir=param_default()['cache_dir'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /workspaces/code-rationales/datax/df_cache_dir/json/default-c7c5494e66867cf3/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
test_dataset = test_dataset['train']
test_dataset[0]

{'n_ast_errors': 0,
 'signature': 'def ping() -> str',
 'prompt': 'Generate Pyhton code that Check if the grid client is up.\n\n    Check for Ping? Get a Pong!\n\n    Returns:\n        str: pong :',
 'complexity': 1,
 'code': 'def ping() -> str:\n    \n\n    return "pong"\n',
 'path': 'packages/grid/backend/grid/api/meta/status.py',
 'fun_name': 'ping',
 'token_counts': 9,
 'language': 'Python',
 'docstring': 'Check if the grid client is up.\n\n    Check for Ping? Get a Pong!\n\n    Returns:\n        str: pong\n    ',
 'n_identifiers': 2,
 'nloc': 9,
 'ast_errors': '',
 'commit_id': 'e92a1b85d7e22328cc525c74c710cb20b880fd21',
 'n_whitespaces': 12,
 'ast_levels': 6,
 'id': 151,
 'commit_message': 'grid: add docstring and tests for grid status api',
 'n_ast_nodes': 20,
 'url': 'https://github.com/OpenMined/PySyft.git',
 'vocab_size': 6,
 'file_name': 'status.py',
 'repo': 'PySyft',
 'n_words': 6}

In [10]:
test_dataset = test_dataset.select(range(param_default()['num_samples']))

In [11]:
###### IMPORTANT MODIFIY 
#df_sampled_code = test_dataset.to_pandas()[['code','prompt']] #code_completion_random_cut_5k_30_512_tokens
df_sampled_code = test_dataset.to_pandas()[['docstring','code','prompt']] # others

In [12]:
df_sampled_code.describe()

,docstring,code,prompt
count,100,100,100
unique,100,100,100
top,Check if the grid client is up.\n\n Check f...,"def ping() -> str:\n \n\n return ""pong""\n",Generate Pyhton code that Check if the grid cl...
freq,1,1,1


## Model Loading and Testing

In [13]:
model = AutoModelForCausalLM.from_pretrained(
            param_default()['model_name'],
            cache_dir=param_default()['cache_dir'])

In [14]:
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32768, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [15]:
model.device

device(type='cuda', index=0)

## Tokenizer Loading and Testing

In [16]:
tokenizer = AutoTokenizer.from_pretrained(param_default()['model_name'])

## Samples Encoding and Filtering

In [17]:
###### IMPORTANT MODIFIY 
#df_sampled_code['ground_truth'] = df_sampled_code['code'] #code_completion_random_cut_5k_30_512_tokens
df_sampled_code['ground_truth'] = df_sampled_code['docstring'] + '\n' + df_sampled_code['code'] #others

In [18]:
df_sampled_code['size'] =  df_sampled_code['ground_truth'].map(lambda code: len(tokenizer(code)['input_ids']))
df_sampled_code['input_ids'] = tokenizer(df_sampled_code['prompt'].tolist())['input_ids']

In [19]:
df_sampled_code = df_sampled_code.drop('docstring', axis=1)
df_sampled_code = df_sampled_code.drop('code', axis=1)

In [20]:
df_sampled_code.head(5)

,prompt,ground_truth,size,input_ids
0,Generate Pyhton code that Check if the grid cl...,Check if the grid client is up.\n\n Check f...,42,"[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3..."
1,Generate Pyhton code that Sign the JWT with RS...,\n Sign the JWT with RSA-256 using the ...,212,"[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1..."
2,Generate Pyhton code that Return the name of t...,Return the name of the coverage group for the ...,221,"[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5..."
3,Generate Pyhton code that Tests run_experiment...,Tests run_experiment on cluster shutdown with ...,63,"[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ..."
4,Generate Pyhton code that Blacklisted but then...,\n Blacklisted but then subsequently wh...,280,"[6864, 1611, 517, 265, 1233, 626, 30593, 15023..."


## Model Sampling Generation

In [21]:
print(df_sampled_code['size'].max())
print(df_sampled_code['size'].median())

403
117.0


In [22]:
SAMPLES = 30 #<---- Hardocoded
MAX_GEN_TOK = df_sampled_code['size'].max()

In [23]:
MAX_GEN_TOK

403

In [24]:
inputs = tokenizer(["def hello_world():"], return_tensors="pt")
inputs.to(device)
outputs = model.generate(**inputs, do_sample=True, max_length=MAX_GEN_TOK, top_k=0, num_return_sequences=2, pad_token_id=tokenizer.eos_token_id)
#tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):])


In [25]:
outputs

tensor([[  318, 16509,    63,  7617,   837,   272,  1449,  6257,  6701,  2344,
           339,   687,  1275,  1018,   492,  2281,  2655,    14,  1974,   492,
         16509,    63,  7617,   272,   702,   440,  2281,  2655,    14,  1974,
            14,  1989,    63,  5006,    63,  7617,   199,   199,   318,  1072,
           837,   272,  1449,  2959,   282,  1253,    13,  2453,   511,  1072,
          1245,  2344,   339,   511,    63,  1462,   275,   511,    63,  1100,
          1061,  1759,    63,  2308,   342,   272,   702,  3355,     8,   396,
            63,  1462,     9,  1137,   971,   267,  3355,     8,   396,    63,
         17679,     9,   503,   822,     8,   396,    63,  1462,     9,   272,
           702,  1072,    63,   908,   480,  2567,   401,   298,  5884,   531,
           508,   298,  5884,    26,  1052,     2,   272,   702,  1072,    63,
          1600,   199,   199,   318,  1324,   837,   272,  1449,  3205,   282,
          3486,  1414,  1072,    14,  3885,   536,  

In [26]:
torch.tensor(outputs.tolist()).to(model.device)

tensor([[  318, 16509,    63,  7617,   837,   272,  1449,  6257,  6701,  2344,
           339,   687,  1275,  1018,   492,  2281,  2655,    14,  1974,   492,
         16509,    63,  7617,   272,   702,   440,  2281,  2655,    14,  1974,
            14,  1989,    63,  5006,    63,  7617,   199,   199,   318,  1072,
           837,   272,  1449,  2959,   282,  1253,    13,  2453,   511,  1072,
          1245,  2344,   339,   511,    63,  1462,   275,   511,    63,  1100,
          1061,  1759,    63,  2308,   342,   272,   702,  3355,     8,   396,
            63,  1462,     9,  1137,   971,   267,  3355,     8,   396,    63,
         17679,     9,   503,   822,     8,   396,    63,  1462,     9,   272,
           702,  1072,    63,   908,   480,  2567,   401,   298,  5884,   531,
           508,   298,  5884,    26,  1052,     2,   272,   702,  1072,    63,
          1600,   199,   199,   318,  1324,   837,   272,  1449,  3205,   282,
          3486,  1414,  1072,    14,  3885,   536,  

In [27]:
def df_sampled_generation(
        df_sampled_code, 
        model,
        number_samples = 1,
        max_gen_tok = 100, 
        top_k = 0
    ):
    dict_generated_code = {i: [] for i in range(number_samples)}
    for prompt in df_sampled_code['prompt']:
        input = tokenizer([prompt], return_tensors="pt")
        input.to(model.device)
        outputs = model.generate(**input, do_sample=True,
                                 max_length=max_gen_tok, 
                                 top_k=top_k, 
                                 num_return_sequences=number_samples, 
                                 pad_token_id=tokenizer.eos_token_id)
        for index, output in enumerate(outputs):
            #dict_generated_code[index].append(output[len(input['input_ids'][0]):].tolist())
            dict_generated_code[index].append(output.tolist())
    df_temp = pd.DataFrame().from_dict(data=dict_generated_code) # DataFrame from Generation
    df_temp = pd.concat([df_sampled_code.reset_index(), df_temp ], axis=1) #Index before concating
    return df_temp
        
    

In [28]:
#WARNING TIME CONSUMING
df_generated_input = df_sampled_generation(
    df_sampled_code=df_sampled_code, 
    model=model, 
    number_samples=SAMPLES, 
    max_gen_tok=MAX_GEN_TOK)

In [29]:
df_generated_input.head(5)

,index,prompt,ground_truth,size,input_ids,0,1,2,3,4,...,20,21,22,23,24,25,26,27,28,29
0,0,Generate Pyhton code that Check if the grid cl...,Check if the grid client is up.\n\n Check f...,42,"[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...",...,"[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3..."
1,1,Generate Pyhton code that Sign the JWT with RS...,\n Sign the JWT with RSA-256 using the ...,212,"[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...",...,"[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1..."
2,2,Generate Pyhton code that Return the name of t...,Return the name of the coverage group for the ...,221,"[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...",...,"[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5..."
3,3,Generate Pyhton code that Tests run_experiment...,Tests run_experiment on cluster shutdown with ...,63,"[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...",...,"[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ..."
4,4,Generate Pyhton code that Blacklisted but then...,\n Blacklisted but then subsequently wh...,280,"[6864, 1611, 517, 265, 1233, 626, 30593, 15023..

In [30]:
df_generated_input[0]

0     [6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...
1     [6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...
2     [6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...
3     [6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...
4     [6864, 1611, 517, 265, 1233, 626, 30593, 15023...
                            ...                        
95    [6864, 1611, 517, 265, 1233, 626, 961, 2295, 6...
96    [6864, 1611, 517, 265, 1233, 626, 437, 2008, 4...
97    [6864, 1611, 517, 265, 1233, 626, 437, 8018, 1...
98    [6864, 1611, 517, 265, 1233, 626, 1379, 25344,...
99    [6864, 1611, 517, 265, 1233, 626, 1432, 314, 1...
Name: 0, Length: 100, dtype: object

### Statistics and Checkpoint

In [31]:
np_len_method = [ (np.array([ len(gen_method) for gen_method in df_generated_input[j] ]).mean(),
                   np.array([ len(gen_method) for gen_method in df_generated_input[j] ]).std()  )
                    for j in range(30) ]

In [32]:
np_len_method

[(403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0),
 (403.0, 0.0)]

In [33]:
#Checkpoint of Generation
def checkpoint_generation( df , name = 'output' ):
    df.to_csv(param_default()['sampling_results'] +  '/' + name + '.csv')
    pass

In [34]:
checkpoint_generation( df = df_generated_input, name=param_default()['dataset'])

In [35]:
df_generated_input = pd.read_csv( param_default()['sampling_results'] + '/' + param_default()['dataset'] +'.csv' , index_col=0)

In [36]:
df_generated_input.head()

,index,prompt,ground_truth,size,input_ids,0,1,2,3,4,...,20,21,22,23,24,25,26,27,28,29
0,0,Generate Pyhton code that Check if the grid cl...,Check if the grid client is up.\n\n Check f...,42,"[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...",...,"[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3...","[6864, 1611, 517, 265, 1233, 626, 2670, 340, 3..."
1,1,Generate Pyhton code that Sign the JWT with RS...,\n Sign the JWT with RSA-256 using the ...,212,"[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...",...,"[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1...","[6864, 1611, 517, 265, 1233, 626, 6632, 314, 1..."
2,2,Generate Pyhton code that Return the name of t...,Return the name of the coverage group for the ...,221,"[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...",...,"[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5...","[6864, 1611, 517, 265, 1233, 626, 1432, 314, 5..."
3,3,Generate Pyhton code that Tests run_experiment...,Tests run_experiment on cluster shutdown with ...,63,"[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...",...,"[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ...","[6864, 1611, 517, 265, 1233, 626, 6496, 1255, ..."
4,4,Generate Pyhton code that Blacklisted but then...,\n Blacklisted but then subsequently wh...,280,"[6864, 1611, 517, 265, 1233, 626, 30593, 15023..

In [37]:
df_generated_input.describe()

,index,size
count,100.00,100.00
mean,49.50,137.99
std,29.01,93.53
min,0.00,29.00
25%,24.75,63.00
50%,49.50,117.00
75%,74.25,194.00
max,99.00,403.00


In [38]:
#tst decoding
decoded_input = tokenizer.decode(eval(df_generated_input['input_ids'][1]))
decoded_output = tokenizer.decode(eval(df_generated_input['1'][1]))
print(decoded_input)
print('-'*100)
print(decoded_output)

Generate Pyhton code that Sign the JWT with RSA-256 using the private key found in service account JSON file. :
----------------------------------------------------------------------------------------------------
Generate Pyhton code that Sign the JWT with RSA-256 using the private key found in service account JSON file. :param "encrypted_jtidy_secret":              Text to be used when generating JWT
    :return: JWT object
    """
from __future__ import print_function

import base64
import datetime
import logging
import sys
import signal
import time

from openflow.core import models
from openflow.provisioner import exceptions
from openflow.provisioner import lambda_opts
from openflow.provisioner.field_yaml.controllers import PfeForcer
from openflow.provisioner.filters import (FilterMissingRule,
    TokenFilterMissingRule)


class SecurityLoggingFilter(FilterMissingRule):
    """Filters the logs from Apache display and for debugging purposes.

    This filters know our password is cor

In [39]:
## MEMORy DEALLOCATION
torch.cuda.empty_cache()